In [ ]:
# importing packages and libaray
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
import importlib
import Ganmodel
from tensorflow.keras.layers import Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input

In [ ]:
# defining path for accessing dataset
image_dir = 'H:/Downloads/keras_png_slices_data/Dataset/keras_png_slices_train/image'

In [ ]:
def display_images(): 
        r, c = 1,4
        noise = np.random.normal(0, 1, (r * c,100)) 
        generated_images = generator.predict(noise) 
  
        #Scaling the generated images 
        generated_images = 0.5 * generated_images + 0.5
      
        imgs = generator.predict(noise)
        fig = plt.figure(figsize=(40,10))
        for i, img in enumerate(generated_images):
            ax = fig.add_subplot(1,4,i+1)
            ax.imshow(img,cmap="gray")
        fig.suptitle("Generated images ",fontsize=30)

In [ ]:
# function for loading images.
def data_gen(img_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    np.random.shuffle(n)
    print("Number of Train_images:",len(n))
    
    while (True):
        img = np.zeros((batch_size, 256, 256, 1)).astype('float')
       
        for i in range(c, c+batch_size): #Reading set of images for a mentioned batch-size . 
            train_img = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(
                img_folder+'/'+n[i],color_mode='grayscale', target_size=(256, 256)))/255
            
            img[i-c] = train_img #add to array - img[0], img[1], and so on.
             
        c+=batch_size
        if(c+batch_size>=len(os.listdir(img_folder))):
            c=0
            np.random.shuffle(n)
                  
        return img

In [ ]:
#Build Generator and Discriminator

discriminator = Ganmodel.make_discriminator_model()

discriminator.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(0.0002,0.5),metrics=['accuracy'])
discriminator.trainable = False

generator = Ganmodel.make_generator_model()

In [ ]:
noise = Input((100,))
#print(noise.shape)
generated_image = generator(noise)

#plt.imshow(generated_image[0, :, :, 0])
generated_image.shape

In [ ]:

#generate image from generator with noise as input
generated_image = generator(noise)

#sending generated image as input for discriminator to discriminate as real(1) or fake(0)
v= discriminator(generated_image)

#combining both models
combined_network = Model(noise, v) 

#compiling combined model with binary_crossentropy loss and Adam optimser with lr=0.0002 and beta=0.5
combined_network.compile(loss='binary_crossentropy', 
                         optimizer=tf.keras.optimizers.Adam(0.0002,0.5)) 


#defining parameter
#noise = Input((100,))
num_epochs=15000
batch_size=16
display_interval=1000
losses=[] 

#Defining the Adversarial ground truths 
valid = np.ones((batch_size, 1)) 

#Adding some noise  
valid += 0.05 * np.random.random(valid.shape) 
fake = np.zeros((batch_size, 1)) 
fake += 0.05 * np.random.random(fake.shape)
#print(fake.shape)

In [ ]:
train_gen = data_gen(image_dir, batch_size = 16)

In [ ]:
for epoch in range(num_epochs): 
    #Sampling a random half of images 
    index = np.random.randint(0, train_gen.shape[0], batch_size) 
    images = train_gen[index]
 
    #Sampling noise and generating a batch of new images 
    noise = np.random.normal(0, 1, (batch_size, 100))
    generated_images = generator.predict(noise) 
              
    #Training the discriminator to detect more accurately 
    #whether a generated image is real or fake 
    discm_loss_real = discriminator.train_on_batch(images, valid) 
    discm_loss_fake = discriminator.train_on_batch(generated_images, fake) 
    discm_loss = 0.5 * np.add(discm_loss_real, discm_loss_fake) 
              
  
    #Training the generator to generate images 
    #which pass the authenticity test 
    genr_loss = combined_network.train_on_batch(noise, valid) 
    
    #Tracking the progress                 
    if epoch % display_interval == 0: 
        display_images() 